For kicks, we're going to build a super simple neural network with only one layer.

- Import libraries and data
- Identify stocks we want to highlight
- These will be stocks that increase in value the next day
- So 'high' higher than previous 'high'
- Create a matrix of our training values
- Define our parameters matrix
- Pass data the data through activation function
- Pass to Loss function
- Back propagation

In [382]:
# First we import libraries and build arrays and tensors
import pandas as pd  # To manage the initial dataframe
import numpy as np  # Prec-process and organize the data
import tensorflow as tf  # Build tensors
import tensorflow.keras as keras  # Compute the activation

data_file = pd.read_csv('C:/Users/alexa/Documents/Code/Jupyter Playground/Apple Stocks/02-data/AppleStockPrices.csv')

data_file.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,1/3/2011,325.64,330.26,324.84,329.57,15883600,329.57
1,1/4/2011,332.44,332.50,328.15,331.29,11038600,331.29
2,1/5/2011,329.55,334.34,329.50,334.00,9058700,334.00
3,1/6/2011,334.72,335.25,332.90,333.73,10709500,333.73
4,1/7/2011,333.99,336.35,331.90,336.12,11096800,336.12


- We don't care about Date or Adj Close
- We will seperate out all those values where the next day is higher
- We will then have one set whose value after applying the sigmoid is 1
- We will also have another set whose value after applying the sigmoid is 0

In [383]:
# Here we build out the arrays and data structures that we need

# Identify the data we care about
wip_data = data_file[['Open', 'High', 'Low', 'Close', 'Volume']]  # The values we will use for the model

# Build our labeled df
High_stop = wip_data['High']  # Value we are testing against
High_start = wip_data['High'].shift(periods=1)  # Starting value to be subtracted

# Create a matrix with high subtracted from next days high
labeled_df = High_stop - High_start  # Values we will label

# We can then convert everything to numpy arrays to work with
ml_data = wip_data.to_numpy()  # Our primary data to numpy
ml_labels_wip = labeled_df.to_numpy()  # Our labels to numpy
ml_labels = np.empty(34, dtype=float)  # An empty array for our final labeling

# Convert the labels array into 0 and 1 labels
ml_labels[ml_labels_wip < 1] = 0   # Label values losing value
ml_labels[ml_labels_wip >= 1] = 1   # Label values gaining value

# Now we will build the data to train on
x_data_product = np.prod(ml_data, axis=1)  # Multiply rows in the main array so we have a single vector

# Normalize the multiplied data (min-max normalization)
min_val = np.min(x_data_product)  # Identify the smallest value
max_val = np.max(x_data_product)  # Identify the largest value
x = (x_data_product - min_val) / (max_val - min_val)  # Calculate the final normalized value

# Create the weights of our array
w = np.random.randn(len(x))  # Create a randomized set of weights for training

# Weighted array
ml_xw = x * w  # Multiply input values by weights (note that we're doing a row by row operation and not the dot product

print(ml_labels.shape)
print(ml_xw.shape)

(34,)
(34,)


In [384]:
# We're going to use the tanh function as our activation function
# We will use keras to do this as it does a lot of the work for us

# Convert our arrays to tensors
k_xw = tf.convert_to_tensor(ml_xw)
y = tf.convert_to_tensor(ml_labels)

# Run the activation function (Hyperbolic Tanh)
y_hat_init = keras.activations.tanh(k_xw)

# We're going to use binary cross entropy for our loss function so need to adjust our initial output
y_hat = (y_hat_init + 1) / 2.0
print(y_hat.shape)
print(y.shape)

(34,)
(34,)


In [385]:
y

<tf.Tensor: shape=(34,), dtype=float64, numpy=
array([nan,  1.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,  0.,
        0.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,
        1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.])>

In [386]:
# Binary cross entropy is defined as-(y\cdot{log}(p_i)+(1-y_i)\cdot{log}(1-p_i))
# We will do the operation manually in np

# Convert tensors to arrays
y_hat_np = tf.keras.ops.convert_to_numpy(y_hat)  # Convert y_hat to np array
y_np = tf.keras.ops.convert_to_numpy(y)  # Convert y to np array

# Removing the first value from each to get rid of the NaN value
y_np[np.isnan(y_np)] = np.mean(y_np[np.logical_not(np.isnan(y_np))])
epsilon = 1e-7
y_hat_np = np.clip(y_hat_np, epsilon, 1 - epsilon) 

print(y_hat_np.shape)
print(y_np.shape)

bce = -np.mean((y_np * np.log(y_hat_np)) + (1 - y_np) * np.log(1 - y_hat_np))
print(bce)

(34,)
(34,)
0.7186066510319156


In [387]:
# Next step will be to run backpropagation on the network to update the weights 

# Make sure everything has a mathing shape

# Derivative of the loss function
loss_grad = (y_hat_np - y_np) / (y_hat_np * (1 - y_hat_np))

# Derivative of the activation function 
activation_grad = 1 - np.tanh(ml_xw) ** 2 

# Learning rate 
learning_rate = 0.001 

# Compute the overall gradient for weights 
overall_grad = loss_grad * activation_grad 

# Update weights  
weights_update = learning_rate * overall_grad * x 
w -= weights_update 

print("Updated Weights:", w)

Updated Weights: [-0.17075729 -2.2733749  -0.18078474  0.61515213 -0.81707099 -0.38942105
  2.41769458  2.45869366 -1.67501045 -0.38708297 -2.27990755 -1.21382289
  0.64120267 -1.1558855   1.73577321 -0.44085734 -0.03333584 -0.55049657
 -1.22485986 -0.04507611 -1.21075773  1.00612847  1.32497119 -0.23640786
  1.23762109  0.68091641  0.18921295 -1.04267805  0.76123833 -1.02813454
  1.73821841 -0.99437929 -0.97030293 -0.81891997]


In [388]:
# Now we will run a couple more epochs to see if the value of our loss decreases
weights = w
values = w * x
actual = y_np
actual[0] = 1
prediction = np.tanh(values)
epsilon = 1e-7  # Small value to avoid log(0)
prediction = np.clip(prediction, epsilon, 1 - epsilon)
loss_calc = -np.mean(actual * np.log(prediction) + (1 - actual) * np.log(1 - prediction))
print("Original Loss: ", bce)
print("Updated Loss: ", loss_calc)

loss_grad_2 = (prediction - actual) / (prediction * (1 - prediction))
activation_grad_2 = 1 - np.tanh(prediction) ** 2
learning_rate_2 = 0.999999  # Updating the learning rate to speed up performance
overall_grad_2 = loss_grad_2 * activation_grad_2
weights_update_2 = learning_rate_2 * overall_grad_2 * x
w -= weights_update_2

values = w * x
actual = y_np
actual[0] = 1
prediction = np.tanh(values)
epsilon = 1e-7  # Small value to avoid log(0)
prediction = np.clip(prediction, epsilon, 1 - epsilon)
loss_calc = -np.mean(actual * np.log(prediction) + (1 - actual) * np.log(1 - prediction))
print("2nd Loss calc: ", loss_calc)

loss_grad_2 = (prediction - actual) / (prediction * (1 - prediction))
activation_grad_2 = 1 - np.tanh(prediction) ** 2
learning_rate_2 = 0.999999  # Updating the learning rate to speed up performance
overall_grad_2 = loss_grad_2 * activation_grad_2
weights_update_2 = learning_rate_2 * overall_grad_2 * x
w -= weights_update_2

values_3 = w * x
prediction = np.tanh(values_3)
epsilon = 1e-7  # Small value to avoid log(0)
prediction = np.clip(prediction, epsilon, 1 - epsilon)
loss_calc = -np.mean(actual * np.log(prediction) + (1 - actual) * np.log(1 - prediction))
print("Final loss value: ", loss_calc)

Original Loss:  0.7186066510319156
Updated Loss:  7.402623444854449
2nd Loss calc:  0.6153725689342333
Final loss value:  0.5937316041235796
